#### Imports

In [8]:
import pandas as pd

from scripts.preprocessing import preprocess_pipeline,split_train_test
from scripts.model import XGBOOSTPredictor

Leitura dos dados

In [9]:
df = pd.read_csv("./bases/PEDE 2024 - DATATHON - PEDE2022.csv")

In [10]:
df.head()

,RA,Fase,Turma,Nome,Ano nasc,Idade 22,Gênero,Ano ingresso,Instituição de ensino,Pedra 20,...,Inglês,Indicado,Atingiu PV,IPV,IAN,Fase ideal,Defas,Destaque IEG,Destaque IDA,Destaque IPV
0,RA-1,7,A,Aluno-1,2003,19,Menina,2016,Escola Pública,Ametista,...,"6,0",Sim,Não,"7,278","5,000",Fase 8 (Universitários),-1,Melhorar: Melhorar a sua entrega de lições de ...,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...
1,RA-2,7,A,Aluno-2,2005,17,Menina,2017,Rede Decisão,Ametista,...,"9,7",Não,Não,"6,778","10,000",Fase 7 (3º EM),0,Melhorar: Melhorar a sua entrega de lições de ...,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...
2,RA-3,7,A,Aluno-3,2005,17,Menina,2016,Rede Decisão,Ametista,...,"6,9",Não,Não,"7,556","10,000",Fase 7 (3º EM),0,Destaque: A sua boa entrega das lições de casa.,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Destaque: A sua boa integração aos Princípios ...
3,RA-4,7,A,Aluno-4,2005,17,Menino,2017,Rede Decisão,Ametista,...,"8,7",Não,Não,"5,278","10,000",Fase 7 (3º EM),0,Melhorar: Melhorar a sua entrega de lições de ...,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...
4,RA-5,7,A,Aluno-5,2005,17,Menina,2016,Rede Decisão,Ametista,...,"5,7",Não,Não,"7,389","10,000",Fase 7 (3º EM),0,Destaque: A sua boa entrega das lições de casa.,Melhorar: Empenhar-se mais nas aulas e avaliaç...,Melhorar: Integrar-se mais aos Princípios Pass...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   RA                     860 non-null    object
 1   Fase                   860 non-null    int64 
 2   Turma                  860 non-null    object
 3   Nome                   860 non-null    object
 4   Ano nasc               860 non-null    int64 
 5   Idade 22               860 non-null    int64 
 6   Gênero                 860 non-null    object
 7   Ano ingresso           860 non-null    int64 
 8   Instituição de ensino  860 non-null    object
 9   Pedra 20               323 non-null    object
 10  Pedra 21               462 non-null    object
 11  Pedra 22               860 non-null    object
 12  INDE 22                860 non-null    object
 13  Cg                     860 non-null    object
 14  Cf                     860 non-null    int64 
 15  Ct                     

A análise exploratória dos dados foi realizada no notebook ``analise_dados_2022`` na pasta notebooks. Alguns dos passos tomados para o pré processamento dessa base foram:

1. Conversão de Colunas Numéricas
Algumas colunas contendo valores numéricos estavam armazenadas como texto com separadores decimais no formato de vírgula. Essas colunas foram convertidas para o tipo numérico, substituindo a vírgula pelo ponto e aplicando a conversão para o formato adequado.

2. **Codificação da Variável Alvo:**
A variável alvo "Atingiu PV", originalmente categórica com valores "Sim" e "Não", foi convertida para um formato binário, onde "Sim" foi mapeado para 1 e "Não" para 0. Além disso, a coluna "IPV", que não era relevante para o modelo, foi removida.

3. **Conversão de Comentários Qualitativos:**
Algumas colunas continham textos qualitativos que indicavam destaques ou pontos de melhoria dos alunos. Para estruturar essa informação numericamente, os valores foram transformados da seguinte forma:

- Comentários que começavam com "Destaque" foram codificados como 1.
- Comentários que começavam com "Melhorar" foram codificados como -1.
- Demais valores foram atribuídos como 0.
- Após a conversão, as colunas originais de texto foram removidas.

4. **Codificação de Variáveis Categóricas:**
A variável "Instituição de Ensino" era categórica e foi transformada em um formato adequado para modelos de machine learning usando a técnica One-Hot Encoding. Essa transformação gerou uma nova matriz com colunas representando cada instituição, substituindo a coluna original.

5. **Renomeação de Colunas**
Para melhorar a legibilidade e padronizar os nomes das variáveis, algumas colunas foram renomeadas. Por exemplo, "Cg" passou a ser "Ranking_Geral", e "IAA" foi renomeado para "Indicador_Auto_Avaliacao".

6. **Divisão dos Dados em Conjunto de Treino e Teste**
Finalmente, os dados foram separados em conjuntos de treino e teste, garantindo uma divisão estratificada para manter a proporção original da variável alvo - isso foi realizado especialmente por causa do desbalanceamento da base, com uma proporção maior de valores 0 do que 1. A divisão foi realizada utilizando a função ``train_test_split``, com 80% dos dados destinados ao treino e 20% para teste, e com a definição de uma semente aleatória (``random_state=42``) para reprodutibilidade.

In [12]:
df = preprocess_pipeline(df)
X_train, X_test, y_train, y_test = split_train_test(df,target_col="Atingiu PV")

### Uso do Modelo XGBoost para Prever a Variável 'Atingiu PV'

A previsão da variável "Atingiu PV" foi tratada como um problema de classificação binária, pois essa variável assume apenas dois valores possíveis:

- Sim (1): O aluno atingiu o Ponto de Virada (PV).
- Não (0): O aluno não atingiu o Ponto de Virada (PV).

Dado que a definição do PV está relacionada ao IPV (Indicador de Ponto de Virada), que mede o desenvolvimento das aptidões dos alunos com base em uma avaliação qualitativa e quantitativa, a previsão desse resultado pode ser útil para antecipar quais alunos podem precisar de apoio extra para atingir o PV.

### Por que utilizar o modelo XGBoost?

O XGBoost (Extreme Gradient Boosting) foi escolhido para essa tarefa porque é um dos modelos de machine learning mais eficazes para problemas de classificação binária devido a fatores como:

Alta Performance: XGBoost utiliza otimizações como paralelização, poda de árvores e regularização, tornando-o eficiente e robusto contra overfitting.
Interpretabilidade: A importância das features pode ser analisada, permitindo entender quais fatores mais influenciam a determinação do PV.

O modelo foi testado junto a diferentes modelos para classificação binária e foi o que apresentou os melhores valores utilizando a métrica F1-Score. A comparação com os outros modelos pode ser vista no notebook ``analise_dados_2022`` na pasta notebooks. 

### Por que modelamos a previsão como um problema de classificação?

A decisão de tratar a previsão do PV como uma classificação binária se baseia na forma como a variável "Atingiu PV" é definida:

- A variável não representa um valor contínuo ou ordinal, mas sim uma decisão binária com base no IPV.
- O IPV pode mudar anualmente, mas a classificação se mantém: os alunos são categorizados como atingindo ou não o PV.
- Métodos de classificação são mais apropriados para prever probabilidades associadas à classe e podem ser interpretados para entender quais fatores contribuem mais para o atingimento do PV.

In [13]:
model = XGBOOSTPredictor()

model.fit(X_train, y_train)

Modelo treinado com sucesso!


In [14]:
model.save("./models/xgboost_model.pkl")

Modelo salvo em ./models/xgboost_model.pkl
